# Azure AI Agents function calling

This version has the SDK poll on our behalf

## Define a function for your agent to call

In [10]:
import json
from typing import Any, Callable, Set, Dict, List, Optional

def fetch_weather(location: str) -> str:
    """
    Fetches the weather information for the specified location.

    :param location (str): The location to fetch weather for.
    :return: Weather information as a JSON string.
    :rtype: str
    """
    # In a real-world scenario, you'd integrate with a weather API.
    # Here, we'll mock the response.
    mock_weather_data = {
        "New York": "Sunny, 25°C", 
        "London": "Cloudy, 18°C", 
        "Tokyo": "Rainy, 22°C"
    }
    weather = mock_weather_data.get(location, "Weather data not available for this location.")
    weather_json = json.dumps({"weather": weather})
    return weather_json

def fetch_restaurant(location: str) -> str:
    """
    Fetches the restaurant information for the specified location.

    :param location (str): The location to fetch the restaurant for.
    :return: Restaurant information as a JSON string.
    :rtype: str
    """
    # In a real-world scenario, you'd integrate with a restaurant API.
    # Here, we'll mock the response.
    mock_restaurant_data = {
        "New York": "Tatiana by Kwame Onwuachi, Katz’s Delicatessen, Peter Luger Steakhouse, Sylvia's, Nathan's Famous", 
        "London": "St. JOHN, Señor Ceviche, Gloria and Circolo Popolare, Normah's, Bouchon Racine", 
        "Tokyo": "Chanko & Wanko Restaurant Asakusa Sumo Club, Sky Restaurant 634 Musashi, Ichiran, Shibuya, Rokkasen Otakibashiidori, Hakushu - Kobe Teppanyaki"
    }
    restaurant = mock_restaurant_data.get(location, "Restaurant data not available for this location.")
    restaurant_json = json.dumps({"restaurant": restaurant})
    return restaurant_json

def fetch_budget() -> str:
    """
    Fetches the budget information for the specified location.
    :return: budget information as a JSON string.
    :rtype: str
    """
    # In a real-world scenario, you'd integrate with a another API.
    # Here, we'll mock the response.
    mock_budget_data = {
        "New York": """
            Budget Travelers: Around $121 per day. This includes staying in hostels, eating at budget restaurants, and using public transportation.
            Mid-Range Travelers: Approximately $324 per day. This covers mid-range hotels, dining at average restaurants, and some paid attractions.
            Luxury Travelers: About $923 per day. This includes luxury hotels, fine dining, and private transportation.
        """, 
        "London": """
            Budget Travelers: Around $75 per day. This includes staying in hostels, cooking your own meals, and using public transport.
            Mid-Range Travelers: Approximately $195 per day. This covers mid-range hotels, dining at average restaurants, and some paid attractions.
            Luxury Travelers: About $517 per day. This includes luxury hotels, fine dining, and private transportation.
        """, 
        "Tokyo": """
            Budget Travelers: Around $100 per day. This includes staying in hostels, eating at budget restaurants, and using public transportation.
            Mid-Range Travelers: Approximately $286 per day. This covers mid-range hotels, dining at average restaurants, and some paid attractions.
            Luxury Travelers: About $908 per day. This includes luxury hotels, fine dining, and private transportation.
        """
    }
    budget_json = json.dumps({"budget": mock_budget_data})
    return budget_json


# Statically defined user functions for fast reference
user_functions: Set[Callable[..., Any]] = {
    fetch_weather, fetch_restaurant, fetch_budget
}

## STEP 1: Create a client and agent

In [ ]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import FunctionTool, ToolSet

# Create an Azure AI Client from a connection string, copied from your Azure AI Foundry project.
# It should be in the format "<HostName>;<AzureSubscriptionId>;<ResourceGroup>;<HubName>"
# Customers need to login to Azure subscription via Azure CLI and set the environment variables

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.environ["PROJECT_CONNECTION_STRING"],
)

# Initialize agent toolset with user functions
functions = FunctionTool(user_functions)
toolset = ToolSet()
toolset.add(functions)

agent = project_client.agents.create_agent(
    model="gpt-4o", 
    name="my-agent", 
    instructions="You are an AI Travel Agent. Use the provided functions to help answer questions.", 
    toolset=toolset
)
print(f"Created agent, ID: {agent.id}")

Created agent, ID: asst_n8vdybC82ifLFGOMFFCZu360


## STEP 2: Create a thread

In [12]:
thread = project_client.agents.create_thread()
print(f"Created thread, ID: {thread.id}")

Created thread, ID: thread_Ox8Slffim9AhWULIBCDUzLgV


## Step 3-6: Helper Function
3. Add a message to the thread
4. Run the Agent
5. Check the Run Status
6. Display the Agent's Response


In [13]:
def run_agent(user_input):  
    # Step 3: Add a message to the thread  
    message = project_client.agents.create_message(
        thread_id=thread.id,
        role="user",
        content=user_input,
    )
    print(f"Created message, ID: {message.id}")

    # Step 4 & 5: Create and process agent run in thread with tools
    run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)

    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Step 6: Display the Agent's Response
    elif run.status == 'completed':
            # Fetch all messages in the thread
            messages = project_client.agents.list_messages(thread_id=thread.id)
            if messages.data:
                agent_message = messages.data[0]  # Get the last assistant message
                print(f"Agent Response: {agent_message.content[0].text.value}") 
            else:
                print("No messages found.")

## Running the agent using the Helper Function

In [14]:
user_input = "Hello, what is the weather in Tokyo?"
run_agent(user_input)

Created message, ID: msg_13TOzgHuf3SYuebxVc6E3Zeg
Agent Response: The weather in Tokyo is currently rainy with a temperature of 22°C.


In [6]:
user_input = "What is the budget there?"
run_agent(user_input)

Created message, ID: msg_Dl22qsmuDujQ78EvRlPoDcUA
Agent Response: In Tokyo, the budget ranges depending on your travel style:

- **Budget Travelers**: Around $100 per day, including stays in hostels, meals at budget restaurants, and public transportation.
- **Mid-Range Travelers**: Approximately $286 per day, covering mid-range hotels, dining at average restaurants, and some paid attractions.
- **Luxury Travelers**: About $908 per day, including luxury hotels, fine dining, and private transportation.


In [7]:
user_input = "Where can I eat in London?"
run_agent(user_input)

Created message, ID: msg_pLqk7oZ1qXz0c7KmbPJd4vvr
Agent Response: In London, you can dine at some great places such as:

- **St. JOHN**
- **Señor Ceviche**
- **Gloria and Circolo Popolare**
- **Normah's**
- **Bouchon Racine**


In [15]:
user_input = "I only have 300 USD. What country can I visit for 4 days?"
run_agent(user_input)

Created message, ID: msg_VRqUaBNP1Udqrvl4CTasxFFI
Agent Response: Based on your $300 budget and aiming for a 4-day trip, here are some options for budget travelers:

1. **London**: It costs around $75 per day for budget travelers (hostels, cooking, public transport). A 4-day trip would total about $300—perfect for your budget.

2. **Tokyo**: It costs around $100 per day for budget travelers (hostels, budget restaurants, public transport). This would exceed your budget at $400 for 4 days, so Tokyo might not fit.

3. **New York**: It costs around $121 per day for budget travelers. A 4-day trip totals $484—well above your $300 budget.

For your budget, London is the most feasible option for a 4-day trip.


## Delete Agent to free up resources

In [16]:
# Delete the agent when done
project_client.agents.delete_agent(agent.id)
print("Deleted agent")

Deleted agent
